# Game Data Acquisition

#### Summary:
1. Importing Dependencies
2. Getting Game Name for top_50 Streamers: <br>
     a. Extract the csv file from previous part <br>
     b. Get the most recent Game ID based on twitch channel name <br>
     c. Get Game Name based on the ID
3. Save data to CSV as streamer_game_data.csv
4. Get top 50 games on Twitch
5. Save top 50 games data to CSV as top_game_ranking.csv

Twitch API: https://dev.twitch.tv/docs/api/

### 1. Importing Dependencies:

In [2]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time
import threading
from api_key import (twitch_id, twitch_token)

### 2. Getting Game Name:
#### a. Extract CSV file for top 50 Streamers

In [2]:
# Pulling top 50 data from csv file in Data
df_top50 = pd.read_csv('Data/top_50_channel.csv')
df_top50.head()

,Channel,Watch time(Minutes),Stream time(minutes),Peak viewers,Average viewers,Followers,Followers gained,Views gained,Partnered,Mature,Language
0,Tfue,3671000070,123660,285644,29602,8938903,2068424,78998587,True,False,English
1,shroud,888505170,30240,471281,29612,7744066,833587,30621257,True,False,English
2,Myth,1479214575,134760,122552,9396,6726893,1421811,37384058,True,False,English
3,pokimane,964334055,56505,112160,16026,5367605,2085831,45579002,True,False,English
4,summit1g,6091677300,211845,310998,25610,5310163,1370184,89705964,True,False,English


#### b. Getting Game ID for each streamers:

In [ ]:
# Setting up base API Calls
# Setting up the URL
url_channel = 'https://api.twitch.tv/helix/search/channels'
headers_channel = {
    'client-id': twitch_id,
    'Authorization': twitch_token,
}

params_channel = (
('query', 'a_seagull'),
)
    
response = requests.get(url_channel, headers=headers_channel, params=params_channel).json()

In [ ]:
# Setting up base API Calls
# Setting up the URL
url_channel = 'https://api.twitch.tv/helix/search/channels'
url_games = 'https://api.twitch.tv/helix/games'

# Setting up the headers
headers_channel = {
    'client-id': twitch_id,
    'Authorization': twitch_token,
}

headers_games = {
    'Authorization': twitch_token,
    'Client-Id': twitch_id,
}

In [ ]:
# Creating game_id_list to hold all most recent Game ID for each streamers
game_id_list = []

# Itterrow to call Twitch API based on twitch channel name (streamer name)
for index, row in df_top50.iterrows():
    params_channel = (
    ('query', row['Channel']),
    )
    
    response = requests.get(url_channel, headers=headers_channel, params=params_channel).json()
    results = response['data']
    # Adding the most recent Game ID for each streamer to the game_id_list
    game_id_list.append(results[0]['game_id'])


In [ ]:
# Adding the Game ID to the Dataframe

# Creating copy of original dataframe
df_streamer_game = df_top50.copy()

# Adding the Game ID to the dataframe
df_streamer_game['Game ID'] = game_id_list
df_streamer_game.head()

#### c. Get Game Name based on Game ID:

In [ ]:
# Cleaning up dataframe to show only needed Columns
df_streamer_game = df_streamer_game[['Channel', 'Average viewers' , 'Followers', 'Game ID']]
df_streamer_game.head()

In [ ]:
# Creating an empty list of Game Name
game_name = []

# Itterrow to call Twitch API to get Game Name based on Game ID
for index, row in df_streamer_game.iterrows():
    params_games = (
    ('id', row['Game ID']),
    )
    
    response = requests.get(url_games, headers=headers_games, params=params_games).json()
    results = response['data']
    
    # If no error, append the game name to the game_name list
    try:
        game_name.append(results[0]['name'])
    
    # If error occured, append 'NA' to the game_name list
    except (KeyError, IndexError):
        game_name.append('NA')


In [ ]:
# Adding the Game Name to the dataframe

# Creating copy of the previous dataframe
df_complete = df_streamer_game.copy()

# Adding the Game Name to the new Dataframe
df_complete['Game Name'] = game_name

df_complete.head()

### 3. Save Data to CSV file:

In [ ]:
df_complete.to_csv('Data/streamer_game_data.csv', index = False)

### 4. Getting top 50 games on Twitch

In [3]:
# Calling the Top Game from Twitch api

headers = {
    'client-id': twitch_id,
    'Authorization': twitch_token,
}

params = (
    ('first', '50'),
)

response = requests.get('https://api.twitch.tv/helix/games/top', headers=headers, params=params)


In [ ]:
# Creating the top 50 games list
top_game_list = []

for games in range(50):
    top_game_list.append(results[games]['name'])

In [ ]:
# Create list of numbers for the rank
Rank = list(range(1,51))

# Create the DataFrame with Game Ranking and Game Name
df_ranking = pd.DataFrame(Rank)
df_ranking['Game Name']=name_list
df_ranking = df_ranking.rename(columns = {'0':'Game Ranking'})

df_ranking.head()

### 5. Save Ranking file to CSV:

In [ ]:
df_ranking.to_csv('Data/top_game_ranking.csv', index = False)